In [ ]:
from typing import  Match, List
import re

class CreateEquation:
  def __init__(self, query : str, order : int, dimension : int):
    self.query = query
    self.order = order
    self.dimension = dimension
  
  def compile(self):
    pass


class CreateSystem():

  def __init__(self, path : str, equations : List[CreateEquation]):
    self.path = path
    self.equations = equations
    self.device = "cpu"
    self.backend = "python"
    self.target_template = "./templates/advecdiff.py"
    self.process_token = self.__get_process_token()

  def using(self, device : str, backend : str):
    self.device = device
    self.backend = backend
    return self
  
  def generate(self):
    with open(self.target_template) as file:
      data = file.read()
      pattern = r'\$\$\[(.*?)\]'
      result = re.sub(pattern, self.__replace_with, data)
      with open(self.path, "w") as out_file:
        out_file.write(result)
  
  ##########################################
  ## Replacer
  ##########################################
  """
  Replacer
  """
  def __get_process_token(self):
    return {
      "backend" : self.__dimension_replace
    }
  
  def __dimension_replace(self):
    return f"Backend is {self.backend}"

  def __replace_with(self, match : Match[str]):
    matched_value = match.group(1)
    if matched_value in self.process_token:
      return self.process_token[matched_value]()
    return ""



path = "./out/out_file.py"
q1 = CreateEquation("delta.u + v = 0", order=1, dimension=2)
s = CreateSystem(path, [
  q1
]).using(device="cpu", backend="numba").generate()

In [13]:
# must specify . or *
# must give a valid syntax
# + | . | * | - | = | d[char][number](string) | string(params) | string | emptySpace(\v\t, space)
#dddddt(E) => d*d*d*d*dt(E) => stringdt[number]()
"C + ddt2(C) + Klaplace(C) + grad(C) = 0"
"C + d.dt2(C) + K.laplace(C) + grad(C) = 0"

diminetion


In [ ]:
import re
from collections import deque

class Tree:
  def __init__(self):
    self.token = ""
    self.left = None
    self.right = None

  def __parse(self, query : str, node):
    operator = re.search(r'=', query)
    if operator:
      s1 = query[0: operator.start()]
      s2 = query[operator.start() + 1:]
      node.token = operator.group(0).strip()
      node.left = self.__parse(s1, Tree())
      node.right = self.__parse(s2, Tree())
      return node
    operator = re.search(r'\+|-', query)
    if operator:
      s1 = query[0: operator.start()]
      s2 = query[operator.start() + 1:]
      node.token = operator.group(0).strip()
      node.left = self.__parse(s1, Tree())
      node.right = self.__parse(s2, Tree())
      return node
    operator = re.search(r'\*|\.', query)
    if operator:
      s1 = query[0: operator.start()]
      s2 = query[operator.start() + 1:]
      node.token = operator.group(0).strip()
      node.left = self.__parse(s1, Tree())
      node.right = self.__parse(s2, Tree())
      return node
    node.token = query.strip()
    return node
  
  def parse(self, query : str):
    self.__parse(query, self)
  
  ##########################################
  ## Replacer
  ##########################################
  def __print_item_in_center(self, item : str, size : int):
    if len(item) > size:
      print(item[:size], end='.')
    else:
      nb_space = size - len(item) // 2
      print(' ' * nb_space, end='')
      print(item, end='')
      print(' ' * (size - nb_space - len(item)), end='')
  
  def __print_branch(self, size : int):
    nb = size // 4
    print('-' * nb, end='')
    print('╭', end='')
    i = nb + 1
    while i + nb + 2 < size:
      if (i == (size - 1) // 2):
        print("┴", end='')
      else:
        print("─", end='')
      i += 1
    print('╮', end='')
    print(' ' * (size - (i + 1)), end='')
  
  def __print_tree(self, tree_height : int):
    level_item = tree_height
    item_len = 4
    buffer_size = pow(2, level_item) * item_len

    level = 1
    depth = 0
    level_type = 1
    level_item = 1

    queue = deque()
    is_empty_node = deque()

    queue.append(self)
    while depth < tree_height:
      if (level_type == 1):
        node = queue.popleft()
        if node :
          self.__print_item_in_center(node.token, buffer_size)
          queue.append(node.left)
          queue.append(node.right)
          is_empty_node.append(True)
        else :
          queue.append(None)
          queue.append(None)
          self.__print_item_in_center(" ", buffer_size)
          is_empty_node.append(False)
        level_item -= 1
        if (level_item == 0) :
          level_type = 2
          level_item = level
          level *= 2
          depth += 1
          print()
      else :
        while level_item != 0:
          is_empty = is_empty_node.popleft()
          if is_empty:
            self.__print_branch(buffer_size)
          else:
            self.__print_item_in_center(" ", buffer_size)
          level_item -= 1
        level_item = level
        level_type = 1
        buffer_size //= 2
        print()

  def tree_height(self, node):
      if node is None:
          return -1
      left_height = self.tree_height(node.left)
      right_height = self.tree_height(node.right)
      return 1 + max(left_height, right_height)

  def print(self):
    h = self.tree_height(self) + 1
    self.__print_tree(h)

tree = Tree()
tree.parse("A . B + c")
tree.print()

                +
----╭──┴──╮     
        A        B


In [6]:
import sympy

a = sympy.sqrt(15)
a

sqrt(15)

In [8]:
import sympy as sp
from sympde import symbols
from sympde.calculus import grad, dot, laplace, div
from sympde.topology import Domain
from sympde.expr import BilinearForm, LinearForm, integral

# Define symbols
x, y = symbols('x y')
u = sp.Function('u')(x, y)
v = sp.TestFunction('v')(x, y)

# Define the domain
domain = Domain('Omega', dim=2)

# Define the PDE equation
equation = BilinearForm((u, v), integral(domain, dot(grad(u), grad(v)) - u*v))

# Solve the PDE
solution = sp.solve(equation == 0, u)

# Print the solution
print("Solution:")
print(solution)



AttributeError: module 'sympy' has no attribute 'TestFunction'

In [15]:
import sympdee.sympde as sp
from sympdee.sympde import symbols
from sympdee.sympde.calculus import grad, dot, laplace, div
from sympdee.sympde.topology import Domain
from sympdee.sympde.expr import BilinearForm, LinearForm, integral

from sympdee.sympde.expr import BilinearForm, LinearForm, integral

# Define symbols
x, y = symbols('x y')
u = sp.Function('u')(x, y)
v = sp.TestFunction('v')(x, y)

# Define the domain
domain = Domain('Omega', dim=2)

# Define the PDE equation
equation = BilinearForm((u, v), integral(domain, dot(grad(u), grad(v)) - u*v))

# Solve the PDE
solution = sp.solve(equation == 0, u)

# Print the solution
print("Solution:")
print(solution)



AttributeError: module 'sympdee.sympde' has no attribute 'TestFunction'

In [14]:
from sympde import Unknown, Constant
from sympde import dx

u = Unknown('u', ldim=1)
alpha = Constant('alpha')

expr = dx(alpha*u) + dx(dx(2*u))


ImportError: cannot import name 'Unknown' from 'sympde' (/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/sympde/__init__.py)

In [ ]:
from manapy import symbol, dt, dx, dy, dz, laplace, grad

class Operation():
  Null = -1
  Laplace = 0
  Grad = 1
  Dx = 2
  Dy = 3
  Dz = 4
  Dt = 5

class NodeToken():
  Variable = 0
  Constant = 1

  def __init__(self):
    pass



class Node():
  Variable = 0
  Constant = 1

  def __init__(self, name : str):
    self.operation = Operation.Null
    self.next = None    
    self.name = name
  
  def laplace():
    pass

  def grad():
    pass

  def dx():
    pass

  def dy():
    pass

  def dz():
    pass

  def dt():
    pass

class Expression():
  def __init__() :
    pass

  def __add__(self, x):


x, y = Variable('x y')
k, l = Constant('k l')

expr = dx(x) + dy(y) + 

In [22]:
class Test():
  def __init__(self, name : str):
    self.name = name

  def __add__(self, x):
    print("Add")
    return x
  
  def __mul__(self, y):
    print("Mul")
    return y

  def __rmul__(self, x):
    print("Right Mull", x)
    print(x)
    return Test(str(x))
  
  def __str__(self):
    return self.name


Test(5)

In [12]:
from collections import deque

def get_math_symbol(symbol_name):
  math_symbols = {
  "alpha": "α",
  "beta": "β",
  "gamma": "γ",
  "delta": "δ",
  "epsilon": "ε",
  "zeta": "ζ",
  "eta": "η",
  "theta": "θ",
  "iota": "ι",
  "kappa": "κ",
  "lambda": "λ",
  "mu": "μ",
  "nu": "ν",
  "xi": "ξ",
  "omicron": "ο",
  "pi": "π",
  "rho": "ρ",
  "sigma": "σ",
  "tau": "τ",
  "upsilon": "υ",
  "phi": "φ",
  }
  return math_symbols.get(symbol_name.lower(), symbol_name)

class Constant():
  def __init__(self, name : str, value):
    self.name = get_math_symbol(name)
    self.value = value
  
  def __str__(self):
    return f"C[{self.name}]"
  

class Variable():
  def __init__(self, name : str):
    self.name = get_math_symbol(name)
  
  def __str__(self):
    return f"V[{self.name}]"

class Operation():
  Laplace = 0
  Grad = 1
  Dx = 2
  Dy = 3
  Dz = 4
  Dt = 5
  Mul = 6
  Add = 7
  
  nb_operations = 12

  def __init__(self, opType : int):
    m = ["Laplace", "Grad", "Dx", "Dy", "Dz", "Dt", "Mul", "Add"]
    self.name = m[opType]
    self.type = opType
    self.order = 1
  
  def augment_order(self):
    self.order += 1
    self.name += str(self.order)

  def __str__(self):
    if self.order == 1:
      return f"OP[{self.name}]"
    else :
      return f"OP2[{self.name}]"
  
  def get_index(self):
    if self.type in [Operation.Dx, Operation.Dy, Operation.Dz, Operation.Dt] and self.order == 2:
      return self.type + 6
    return self.type

class Node():

  def __init__(self, value, name = None):
    if isinstance(value, (int, float)):
      if name == None:
        name = str(round(value, 3))
      value = Constant(name, value)
    if isinstance(value, str):
      value = Variable(value)
    if not isinstance(value, (Operation, Variable, Constant)):
      raise TypeError(f"can't construct Node from {type(value)}")
    self.token = value
    self.right = None
    self.left = None

  """
    raise an error if we can't do any operations.
    return  True to add a new node, to have the new node point to self node
            False to alter self content
  """
  def __checkOperation(self, operation : Operation):
    if isinstance(self.token, Variable) :
      return False
    if isinstance(self.token, Operation) :
      if self.token.type == operation.type:
        if self.token.type in [Operation.Laplace, Operation.Grad]:
          raise RuntimeError("Laplace or Grad Can't be nested")
        elif self.token.type in [Operation.Dt, Operation.Dx, Operation.Dy, Operation.Dz] \
          and self.token.order > 1:
          raise RuntimeError("Max order of a variable is 2")
        self.token.augment_order()
        return True

    raise TypeError(f"operation can only perform on Variable type or Node with the same operation type")

  def __addOperation(self, operation : Operation):
    exist = self.__checkOperation(operation)
    if not exist:
      res = Node(operation)
      res.left = self
      res.right = None
      return res
    else:
      return self

  def laplace(self):
    return self.__addOperation(Operation(Operation.Laplace))

  def grad(self):
    return self.__addOperation(Operation(Operation.Grad))

  def dx(self):
    return self.__addOperation(Operation(Operation.Dx))

  def dy(self):
    return self.__addOperation(Operation(Operation.Dy))
  
  def dz(self):
    return self.__addOperation(Operation(Operation.Dz))

  def dt(self):
    return self.__addOperation(Operation(Operation.Dt))

  def exec(self):
    if isinstance(self.token, Operation):
      if self.token.type == Operation.Add:
        return f"{self.left.exec()} + {self.right.exec()}"
      elif self.token.type == Operation.Mul:
        return f"({self.left.exec()}) * ({self.right.exec()})"
      elif self.token.type == Operation.Dt:
        return f"Dt{self.token.order}({self.left.exec()})"
      elif self.token.type == Operation.Dx:
        return f"Dx{self.token.order}({self.left.exec()})"
      elif self.token.type == Operation.Dy:
        return f"Dy{self.token.order}({self.left.exec()})"
      elif self.token.type == Operation.Dz:
        return f"Dz{self.token.order}({self.left.exec()})"
      elif self.token.type == Operation.Laplace:
        return f"Laplace({self.left.exec()})"
      elif self.token.type == Operation.Grad:
        return f"Grad({self.left.exec()})"
    elif isinstance(self.token, (Variable, Constant)):
      return self.token.name
    return ""
  
  def expand(self):
    if isinstance(self.token, Operation):
      if self.token.type == Operation.Add:
        return self.left.expand() + self.right.expand()
      elif self.token.type == Operation.Mul:
        a = self.left.expand()
        b = self.right.expand()
        res = []
        for i in a:
          for j in b:
            z = i + j
            res.append(z)
        return res
    return [[self]]
      

  def __mul__(self, x):
    if not isinstance(x, Node):
      x = Node(x)
    if isinstance(x.token, Constant) and x.token.value == 1:
      return self
    if isinstance(self.token, Constant) and self.token.value == 1:
      return x
    res = Node(Operation(Operation.Mul))
    res.left = self
    res.right = x
    return res

  def __rmul__(self, x):
    x = Node(x)
    return x * self

  def __add__(self, x):
    if not isinstance(x, Node):
      x = Node(x)
    res = Node(Operation(Operation.Add))
    res.left = self
    res.right = x
    return res

  def __radd__(self, x):
    x = Node(x)
    return x + self

  # region Printing
    ##########################################
    ## Printing
    ##########################################


  def __print_item_in_center(self, item : str, size : int):
    if len(item) > size:
      print(item[:size], end='.')
    else:
      nb_space = (size - len(item)) // 2
      print(' ' * nb_space, end='')
      print(item, end='')
      print(' ' * (size - nb_space - len(item)), end='')
  
  def __print_branch(self, size : int):
    nb = size // 4
    print(' ' * nb, end='')
    print('╭', end='')
    i = nb + 1
    while i + nb + 2 < size:
      if (i == (size - 1) // 2):
        print("┴", end='')
      else:
        print("─", end='')
      i += 1
    print('╮', end='')
    print(' ' * (size - (i + 1)), end='')
  
  def __print_tree(self, tree_height : int):
    level_item = tree_height
    item_len = 3
    buffer_size = pow(2, level_item) * item_len

    level = 1
    depth = 0
    level_type = 1
    level_item = 1

    queue = deque()
    is_empty_node = deque()

    queue.append(self)
    while depth < tree_height:
      if (level_type == 1):
        node = queue.popleft()
        if node :
          self.__print_item_in_center(str(node.token), buffer_size)
          queue.append(node.left)
          queue.append(node.right)
          is_empty_node.append(True)
        else :
          queue.append(None)
          queue.append(None)
          self.__print_item_in_center(" ", buffer_size)
          is_empty_node.append(False)
        level_item -= 1
        if (level_item == 0) :
          level_type = 2
          level_item = level
          level *= 2
          depth += 1
          print()
      else :
        while level_item != 0:
          is_empty = is_empty_node.popleft()
          if is_empty:
            self.__print_branch(buffer_size)
          else:
            self.__print_item_in_center(" ", buffer_size)
          level_item -= 1
        level_item = level
        level_type = 1
        buffer_size //= 2
        print()

  def tree_height(self, node):
      if node is None:
          return -1
      left_height = self.tree_height(node.left)
      right_height = self.tree_height(node.right)
      return 1 + max(left_height, right_height)

  # endregion

  def print(self):
    h = self.tree_height(self) + 1
    self.__print_tree(h)
  
  def __str__(self):
    return str(self.token)
  
  def __repr__(self):
    return str(self.token.name)

a = Node("a")
b = Node("b")
c = Node("c")
d = Node("d")
expr = a.dx() + 2 * a.dy() * 3 + a.dz() + b * a.dx() + c * d * 2 * a.dx() + c * d * 2 * a.dx() + Node(1) * 9

expr.print()
display(expr.expand())
print(expr.exec())
arr = expr.expand()




[[Dx], [2, Dy, 3], [Dz], [b, Dx], [c, d, 2, Dx], [c, d, 2, Dx], [9]]

Dx1(a) + ((2) * (Dy1(a))) * (3) + Dz1(a) + (b) * (Dx1(a)) + (((c) * (d)) * (2)) * (Dx1(a)) + (((c) * (d)) * (2)) * (Dx1(a)) + 9


In [13]:
def simplify_expression(expr):
  res = {}
  b = 0
  arr = expr.expand()
  for add in arr:
    val = 1
    var = None
    op = None
    for mul in add:
      if isinstance(mul.token, Constant):
        val *= mul.token.value
      elif isinstance(mul.token, (Variable)):
        var = mul
      elif isinstance(mul.token, (Operation)):
        var = mul.left
        op = mul.token
    if var == None:
      b += val
    else:
      if var not in res:
        res[var] = [0] * (Operation.nb_operations + 1)
      if op == None:
        res[var][0] += val
      else:
        res[var][op.get_index() + 1] += val
  return (res, b)

def simplify_for_op(arr, sub, to):
  items = [arr[i + 1] for i in sub]
  m = min(items)
  for i in sub:
    arr[i + 1] -= m
  arr[to] += m
  return arr

def simplify_for_grad(arr):
  sub = [Operation.Dx, Operation.Dy, Operation.Dz]
  return simplify_for_op(arr, sub, Operation.Grad + 1)

def simplify_for_laplace(arr):
  sub = [Operation.Dx + 6, Operation.Dy + 6, Operation.Dz + 6]
  return simplify_for_op(arr, sub, Operation.Laplace + 1)

def to_tree(theTuple):
  dic, v = theTuple

  tree = None

  tree_adder = lambda x : x if tree == None else tree + x

  if (v != 0):
    tree = tree_adder(Node(v))
    
  for item in dic:
    arr = dic[item]
    arr = simplify_for_grad(arr)
    arr = simplify_for_laplace(arr)
    print(item, "=>", arr)
    for i in range(len(arr)):
      val = arr[i]
      if val != 0:
        val = Node(val)
        if i == 0:
          tree = tree_adder((val * item))
        elif i == Operation.Laplace + 1:
          tree = tree_adder((val * item.laplace()))
        elif i == Operation.Grad + 1:
          tree = tree_adder((val * item.grad()))
        elif i == Operation.Dx + 1:
          tree = tree_adder((val * item.dx()))
        elif i == Operation.Dy + 1:
          tree = tree_adder((val * item.dy()))
        elif i == Operation.Dz + 1:
          tree = tree_adder((val * item.dz()))
        elif i == Operation.Dt + 1:
          tree = tree_adder((val * item.dt()))
        elif i == Operation.Dx + 7:
          tree = tree_adder((val * item.dx().dx()))
        elif i == Operation.Dy + 7:
          tree = tree_adder((val * item.dy().dy()))
        elif i == Operation.Dz + 7:
          tree = tree_adder((val * item.dz().dz()))
        elif i == Operation.Dt + 7:
          tree = tree_adder((val * item.dt().dt()))
  return tree

a = Node("a")
b = Node("b")
c = Node("c")
d = Node("d")
#expr = a.dx() + a.dx().dx() + 2 * Node(1) + 3 * b + a.dy().dy() + a.dz().dz() + b + 6 + a.dy() + a.dz() + 3 * a.grad()
expr = 2 * a.dx() + a.dy() + a.dz() + b.dy() + 3 * a.dx().dx() + 2 * (a.dy().dy() + a.dz().dz()) + b.dx() + b.dz()

print(expr.exec())
print(expr.expand())
tup = simplify_expression(expr)
print(tup)
# simplify_for_grad(arr[0][a])
# simplify_for_laplace(arr[0][a])
new_tree = to_tree(tup)
new_tree.print()
new_tree.exec()

(2) * (Dx1(a)) + Dy1(a) + Dz1(a) + Dy1(b) + (3) * (Dx2(a)) + (2) * (Dy2(a) + Dz2(a)) + Dx1(b) + Dz1(b)
[[2, Dx], [Dy], [Dz], [Dy], [3, Dx2], [2, Dy2], [2, Dz2], [Dx], [Dz]]
({a: [0, 0, 0, 2, 1, 1, 0, 0, 0, 3, 2, 2, 0], b: [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}, 0)
V[a] => [0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
V[b] => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                                                                                                                                                                            OP[Add]                                                                                                                                                                                             
                                                                                                ╭──────────────────────────────────────────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────

'(2) * (Laplace(a)) + Grad(a) + Dx1(a) + Dx2(a) + Grad(b)'

In [118]:
class Symbol():
  def __init__(self, value, name : str = None):
    if not (isinstance(value, (str, int, float))):
      raise TypeError("Type error expect str, int, or float")
    if name != None and not isinstance(name, str):
      raise TypeError("Type error name must be string")
    self.node = Node(value, name)
  
  def laplace(self):
    self.node = self.node.laplace()
    return self

  def grad(self):
    self.node = self.node.grad()
    return self

  def dx(self):
    self.node = self.node.dx()
    return self

  def dy(self):
    self.node = self.node.dy()
    return self
  
  def dz(self):
    self.node = self.node.dz()
    return self

  def dt(self):
    self.node = self.node.dt()
    return self
  
  def print(self):
    self.node.print()
  
  def exec(self):
    print(self.node.exec())

  def __mul__(self, x):
    if not isinstance(x, Symbol):
      x = Symbol(x)
    self.node = self.node * x.node
    return self

  def __add__(self, x):
    if not isinstance(x, Symbol):
      x = Symbol(x)
    self.node = self.node + x.node
    return self

  # 3 + Symbol => Symbol(3) + Symbol
  def __rmul__(self, x):
    x = Symbol(x)
    return x * self

  def __radd__(self, x):
    x = Symbol(x)
    return x + self


def grad(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.grad()

def laplace(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.laplace()

def dx(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.dx()

def dy(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.dy()

def dz(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.dz()

def dt(s : Symbol):
  if not isinstance(s, Symbol):
    s = Symbol(s)
  return s.dt()

expr = (2 * Symbol("a") + Symbol("b")) * (Symbol(5) + Symbol(2)) #=> 2 * grad("a")
expr.print()
expr.exec()

                    OP[Mul]                     
            ╭──────────┴──────────╮             
        OP[Add]                 OP[Add]         
      ╭────┴────╮             ╭────┴────╮       
  OP[Mul]       V[b]        C[5]        C[2]    
   ╭─┴─╮       ╭─┴─╮       ╭─┴─╮       ╭─┴─╮    
 C[2]  V[a]                                     
(((2 * a) + b) * (5 + 2))


In [121]:
expr = (3 * laplace("x")) * (dy("u") * dx("b"))
expr.print()
expr.exec()
expr.node.expand()

                    OP[Mul]                     
            ╭──────────┴──────────╮             
        OP[Mul]                 OP[Mul]         
      ╭────┴────╮             ╭────┴────╮       
    C[3]    OP[Laplace]    OP[Dy]      OP[Dx]   
   ╭─┴─╮       ╭─┴─╮       ╭─┴─╮       ╭─┴─╮    
             V[x]        V[u]        V[b]       
((3 * Laplace(x)) * (Dy(u) * Dx(b)))


[[3, Laplace, Dy, Dx]]

 V[a] 
